<a href="https://colab.research.google.com/github/silverstar0727/ML-Pipeline-Tutorial/blob/main/tfx-pipeline-tutorial/end2end_tfx_pipeline/end2end_tfx_pipeline(Lab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Written by [Hasan Rafiq](https://www.linkedin.com/in/sam04/) / [youtube](https://www.youtube.com/watch?v=VrBoQCchJQU&list=PLlYCzXajMuOl3iUQrsvDn4i-1WkPQTSjX&index=30)
* Translated & Modified by [Jeongmin-Do](https://github.com/silverstar0727)

# 라이브러리 임포트

In [ ]:
# 반드시 실행 후 런타임을 재시작 해주세요
!pip install -q tfx

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 829kB 4.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [ ]:
import os
import pprint
import numpy as np
import tempfile
import urllib

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()
from typing import Dict, List, Text

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [ ]:
# 버전 확인
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.4.1
TFX version: 0.29.0


In [ ]:
# 기존 디렉토리 삭제
!rm -rf data.*
!rm -rf *trainer.py
!sudo rm -r /content/tfx

In [ ]:
# 새로운 디렉토리 추가
! cd /content/
! mkdir /content/tfx/
! mkdir /content/tfx/data
! mkdir /content/tfx/serving_model

In [ ]:
# 시카고 택시 데이터 셋 다운로드
!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-04-30 00:08:15--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.83M  --.-KB/s    in 0.07s   

2021-04-30 00:08:15 (26.6 MB/s) - ‘data.csv’ saved [1922812/1922812]



# 데이터 기초 전처리

../data.csv를 이용하여 

* evaluation 데이터는 ../eval.csv에 
* training data는 ../tfx/data/data_trans.csv에 저장

In [ ]:
df = pd.read_csv('/content/data.csv')

# 사용하지 않는 칼럼 삭제
df = df.drop(['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)

# 결측치 제거
df = df.dropna()

# 마스킹을 통해 train, validation dataset 분할
np.random.seed(seed=2)
msk = np.random.rand(len(df)) < 0.9
traindf = df[msk]
evaldf = df[~msk]

print(len(traindf))
print(len(evaldf))

traindf.to_csv("/content/tfx/data/data_trans.csv", index=False, header=True)
evaldf.to_csv("eval.csv", index=False, header=False)

13077
1442


## 경로변수 지정

In [ ]:
# 경로변수 지정
_tfx_root = os.path.join(os.getcwd(), 'tfx')
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db')
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_root = os.path.join(_tfx_root, 'data')
_data_filepath = os.path.join(_data_root, "data_trans.csv")

_input_fn_module_file = 'inputfn_trainer.py'
_constants_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

# InterativeContext를 이용한 개별 components 실행

## InteractiveContext 인스턴스 생성

InteractiveContext는 TFX에서 제공되는데, 상호작용 방식으로 각 컴포넌트들을 실행시키고 확인할 수 있도록 해줍니다.

In [ ]:
# InterativeContext 인스턴스를 생성하세요
context = ----

## ExampleGen

ExampleGen 컴포넌트는 TFX Pipeline의 시작점으로 다음의 역할을 수행합니다.

1. 데이터 분할($2\over3$ training + $1\over3 $ eval)
    * training data: /content/tfx/CsvExampleGen/examples/1/Split-train/
    * evaluation data: /content/tfx/CsvExampleGen/examples/1/Split-eval/
2. TFRcord형태로 데이터 변환

In [ ]:
# csvexamplegen을 만들고 interactive context를 이용하여 실행하세요
example_gen = ----
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/content/tfx/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:907007,xor_checksum:1619741297,sum_checksum:1619741297"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## StatisticsGen

StatisticsGen은 기초 통계량을 생성하는 역할을 수행합니다.
* training statistics: /content/tfx/StatisticsGen/statistics/2/Split-train/
* evaluation statistics: /content/tfx/StatisticsGen/statistics/2/Split-eval/

In [ ]:
# statisticsgen을 만들고 interactive context를 이용하여 실행하세요
statistics_gen = ----
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/content/tfx/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# 기초 통계량을 시각화 하세요
context.show(----)

## SchemaGen

SchemaGen은 feature에 대한 유형, 규제 등을 정의하는 스키마를 생성하는 컴포넌트입니다.

In [ ]:
# schemagen을 만들고 interactive context를 이용하여 실행하세요
schema_gen = SchemaGen()
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/content/tfx/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# 생성된 스키마를 시각화하세요
context.show(----)

,Type,Presence,Valency,Domain
Feature name,,,,
'dropoff_latitude',FLOAT,required,single,-
'dropoff_longitude',FLOAT,required,single,-
'fare',FLOAT,required,single,-
'pickup_latitude',FLOAT,required,single,-
'pickup_longitude',FLOAT,required,single,-
'trip_start_day',INT,required,single,-
'trip_start_hour',INT,required,single,-
'trip_start_month',INT,required,single,-


## ExampleValidator

ExampleValidator 컴포넌트는 앞서 생성된 기초통계량과 스키마를 바탕으로 데이터의 이상(anomalies)에 대해서 점검합니다.

In [ ]:
# example validator를 만들고 interactive context를 이용하여 실행하세요
example_validator = ----
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/content/tfx/ExampleValidator/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# 생성된 anomalies를 시각화하세요
context.show(----)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Trainer

### inputfn_trainer.py & constants_trainer.py
데이터를 전처리하기 위한 파이썬 파일

In [ ]:
!wget https://github.com/silverstar0727/kfp-tutorial-on-gcp/releases/download/tfx-tutorial/inputfn_trainer.py -O inputfn_trainer.py -q
!wget https://github.com/silverstar0727/kfp-tutorial-on-gcp/releases/download/tfx-tutorial/constants_trainer.py -O constants_trainer.py -q

### model_trainer.py
케라스를 이용한 모델 구성, 검증, 저장의 함수가 존재하는 파이썬 파일

In [ ]:
!wget https://github.com/silverstar0727/kfp-tutorial-on-gcp/releases/download/tfx-tutorial/model_trainer.py -O model_trainer.py -q

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=example_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(),
    custom_config=({"epochs": 1})
    )

context.run(trainer)

2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pickup_latitude (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
dropoff_latitude (InputLayer)   [(None, 1)]          0                                            
___________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['fare'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


100/100 [==============================] - 20s 175ms/step - loss: 289.6795 - rmse: 16.8845 - val_loss: 896.9946 - val_rmse: 29.9495
INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/5/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/content/tfx/Trainer/model/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/content/tfx/Trainer/model_run/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Pusher
_serving_model_dir로 모델을 내보내는 파이프라인의 마지막 컴포넌트

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 6
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 7
        type_id: 16
        uri: "/content/tfx/Pusher/pushed_model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/content/tfx/serving_model/1619535005"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1619535005"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

# 전체 파이프라인

다음의 오케스트레이터를 통해 파이프라인을 실행할 수 있음.
1. Local
2. Airflow
3. Kubeflow

## 초기화

In [ ]:
!rm -rf data.*
!rm -rf *.csv
!sudo rm -r /content/tfx

! cd /content/
! mkdir /content/tfx/
! mkdir /content/tfx/data
! mkdir /content/tfx/serving_model

! mkdir /content/train_data/
! mkdir /content/eval_data/

!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

mkdir: cannot create directory ‘/content/train_data/’: File exists
mkdir: cannot create directory ‘/content/eval_data/’: File exists
--2021-04-27 14:50:07--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.83M  --.-KB/s    in 0.1s    

2021-04-27 14:50:07 (15.2 MB/s) - ‘data.csv’ saved [1922812/1922812]



## 기초 전처리

In [ ]:
df = pd.read_csv('/content/data.csv')

# 사용하지 않는 칼럼 삭제
df = df.drop(['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)

# 결측치 제거
df = df.dropna()

# 마스킹을 통해 train, validation dataset 분할
np.random.seed(seed=2)
msk = np.random.rand(len(df)) < 0.9
traindf = df[msk]
evaldf = df[~msk]

print(len(traindf))
print(len(evaldf))

traindf.to_csv("/content/train_data/data.csv", index=False, header=True)
evaldf.to_csv("/content/eval_data/eval.csv", index=False, header=False)

13077
1442


## 파이프라인 구성

In [ ]:
def create_final_pipeline(
    pipeline_name: Text,
    root_path: Text,
    data_path: Text,
    training_params: Dict[Text, Text],
) -> pipeline.Pipeline:

  # 경로변수 지정
  _metadata_db_root = os.path.join(root_path, 'metadata.db')
  _serving_model_dir = os.path.join(root_path, 'serving_model')

  # 컴포넌트 구성
  example_gen = CsvExampleGen(input=external_input(data_path))

  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  infer_schema = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)

  validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=infer_schema.outputs['schema'])

  trainer = Trainer(
      module_file=os.path.abspath(_model_trainer_module_file),
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=example_gen.outputs['examples'],
      train_args=trainer_pb2.TrainArgs(),
      eval_args=trainer_pb2.EvalArgs(),
      custom_config=(training_params)
      )

  pusher = Pusher(
      model=trainer.outputs['model'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=_serving_model_dir)))

  return pipeline.Pipeline(
    pipeline_name = pipeline_name,                                              # 파이프라인 이름
    pipeline_root =root_path,                                                   # 파이프라인 경로
    components=[
        example_gen, statistics_gen, infer_schema, validate_stats,
        trainer, pusher
    ],                                                                          # 내부 컴포넌트 구성
    metadata_connection_config = metadata.sqlite_metadata_connection_config(_metadata_db_root), # metadata연결구성
    enable_cache=True,
    beam_pipeline_args=['--direct_num_workers=%d' % 0],
  )

## 파이프라인 실행

In [ ]:
# 경로 지정
_tfx_root = os.path.join(os.getcwd(), 'tfx')

# 훈련 파라미터 설정
training_params = {"epochs": 1}

# 파이프라인
pipeline = create_final_pipeline(root_path = _tfx_root,                         # 경로
                           pipeline_name="local_pipeline",                      # 파이프라인 이름
                           data_path="/content/train_data",                     # 데이터 경로
                           training_params=training_params                      # 훈련 파라미터
                           ) 

# local에서 파이프라인 실행
from tfx.orchestration.local.local_dag_runner import LocalDagRunner
LocalDagRunner().run(pipeline)

2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pickup_latitude (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
dropoff_latitude (InputLayer)   [(None, 1)]          0                                            
_________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['fare'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


100/100 [==============================] - 20s 173ms/step - loss: 395.1059 - rmse: 18.8777 - val_loss: 931.7473 - val_rmse: 30.5244
INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/3/Format-Serving/assets
